In [31]:
# A six-sided die, with numbers written on each of its faces, is placed on the 6-by-6 grid above, 
# in the lower-left (yellow) corner. It then makes a sequence of “moves”. 
# Each move consists of tipping the die into an orthogonally adjacent square within the grid.

# The die starts with a “score” of 0. On the Nth move, 
# its score increases by N times the value of the die facing up after the move. 

# However, the die is only allowed to move into a square if its score after the move matches the value in the square. 
# Also, the die cannot be translated or rotated in place in addition to these moves.

# After some number of moves the die arrives in the upper-right (blue) corner.

# The answer to this puzzle is the sum of values in the unvisited squares from the die’s journey.

In [32]:
numbers = [[57,33,132,268,492,732],
           [81,123,240,443,353,508],
           [186,42,195,704,452,228],
           [-7,2,357,452,317,395],
           [5,23,-4,592,445,620],
           [0,77,32,403,337,452]]

In [33]:
# if movement on x-axis 
def get_new_dice_face_config(die_faces_configuration, die_touching_surface_coordi, new_coordi_for_possible_values):

    horizontal = die_faces_configuration[0].copy()
    vertical = die_faces_configuration[1].copy()
    
    previous_positionX = die_touching_surface_coordi[0]
    previous_positionY = die_touching_surface_coordi[1]

    current_positionX = new_coordi_for_possible_values[0]
    current_positionY = new_coordi_for_possible_values[1]
    
    new_horizontal = []
    new_vertical = []  
    
    if previous_positionY > current_positionY:
        for i in horizontal:
            if i == horizontal[-1]:
                new_horizontal.insert(0, i)
            else:
                new_horizontal.insert((horizontal.index(i) + 1), i)
        horizontal = new_horizontal
        vertical[1] = horizontal[1] 
        
    elif previous_positionY < current_positionY:
        for i in horizontal:
            new_horizontal.insert((horizontal.index(i) - 1), i)
        horizontal = new_horizontal
        vertical[1] = horizontal[1] 


    elif previous_positionX < current_positionX:
        for i in vertical:
            new_vertical.insert((vertical.index(i) - 1), i)
        vertical = new_vertical
        horizontal[1] = vertical[1]
        vertical_pop = vertical.pop(-1)
        horizontal_pop = horizontal.pop(-1)
        horizontal.append(vertical_pop)
        vertical.append(horizontal_pop)

         

    elif previous_positionX > current_positionX:
        for i in vertical:
            if i == vertical[-1]:
                new_vertical.insert(0, i)
            else:
                new_vertical.insert((vertical.index(i) + 1), i)
        vertical = new_vertical
        horizontal[1] = vertical[1]
        vertical_pop = vertical.pop(0)
        horizontal_pop = horizontal.pop(-1)
        horizontal.append(vertical_pop)
        vertical.insert(0, horizontal_pop)

        
    if len(set(horizontal)) < 4:
        return horizontal
    if len(set(vertical)) < 3:
        return vertical
    new_die_faces_configuration = [horizontal, vertical]
    
    

    return new_die_faces_configuration

In [34]:
def get_possible_values_from_coordi(x,y):
    possible_values = {}
    if x+1 <= 5:
        possible_values[str(numbers[x+1][y])+'p1'] = [x+1, y]
    if y+1 <= 5:    
        possible_values[str(numbers[x][y+1])+'p2'] = [x, y+1]
    if x-1 >= 0:
        possible_values[str(numbers[x-1][y])+'p3'] = [x-1, y]
    if y-1 >= 0:
        possible_values[str(numbers[x][y-1])+'p4'] = [x, y-1]
    return possible_values       

In [35]:
def get_opposite_face(current_touching_face):
    if current_touching_face == 'n1':
        return 'n3'
    if current_touching_face == 'n2':
        return 'n4'
    if current_touching_face == 'n3':
        return 'n1'
    if current_touching_face == 'n4':
        return 'n2'
    if current_touching_face == 'n5':
        return 'n6'
    if current_touching_face == 'n6':
        return 'n5'

In [36]:
inital_values_of_faces = {'n1':None, 'n2':None, 'n3':None, 'n4':None, 'n5':None, 'n6':None}

In [37]:
parent_list = []
tracking_list = []
die_faces_configuration = [['n4','n1','n2','n3'], ['n5', 'n1', 'n6']]
die_touching_surface_face = die_faces_configuration[1][1]


In [38]:
variable_for_each_iteration = {'current_move':0, 'current_sum':0, 'die_touching_surface_coordi':[5,0], 'die_touching_surface_face':'n1', 'die_faces_configuration':die_faces_configuration, 'current_numbers_assigned_to_faces':inital_values_of_faces, 'previous_positions_visited':[[5,0]]}
parent_list.append(variable_for_each_iteration)

In [39]:
count = 0
while True and count <= 50:
    
    count+=1
    child_list = []
    for i in parent_list:
#       Unpacking
        current_move = i['current_move']
        current_sum = i['current_sum']
        die_touching_surface_coordi = i['die_touching_surface_coordi']
        die_faces_configuration = i['die_faces_configuration']
        current_numbers_assigned_to_faces = i['current_numbers_assigned_to_faces']
        previous_positions_visited = i['previous_positions_visited']

        possible_values = get_possible_values_from_coordi(die_touching_surface_coordi[0], die_touching_surface_coordi[1])
#       possible_values = numbers:coordinates 
        for values in possible_values:
                new_coordinates_for_die_might_be = possible_values[values]
                new_die_faces_configuration = get_new_dice_face_config(die_faces_configuration, die_touching_surface_coordi, new_coordinates_for_die_might_be)
                new_die_touching_surface_face = new_die_faces_configuration[0][1]
                
                opposite_face = get_opposite_face(new_die_touching_surface_face)
                new_current_move = current_move + 1
                new_current_numbers_assigned_to_faces = current_numbers_assigned_to_faces.copy()

                if new_current_numbers_assigned_to_faces[opposite_face] == None:
                        new_current_numbers_assigned_to_faces[opposite_face] = (int(values[:-2])-current_sum)/new_current_move 
                new_current_sum = current_sum + new_current_numbers_assigned_to_faces[opposite_face]*new_current_move
                
                if new_current_sum == int(values[:-2]):
                    if new_current_sum == 732:
                        new_previous_positions_visited = previous_positions_visited.copy()
                        new_previous_positions_visited.append(new_coordinates_for_die_might_be)   
                        child_list.append({'current_move':new_current_move, 'current_sum':new_current_sum, 'die_touching_surface_coordi':new_coordinates_for_die_might_be, 'die_faces_configuration':new_die_faces_configuration, 'current_numbers_assigned_to_faces':new_current_numbers_assigned_to_faces, 'previous_positions_visited':new_previous_positions_visited})
                        tracking_list.append({'current_move':new_current_move, 'current_sum':new_current_sum, 'die_touching_surface_coordi':new_coordinates_for_die_might_be, 'die_faces_configuration':new_die_faces_configuration, 'current_numbers_assigned_to_faces':new_current_numbers_assigned_to_faces, 'previous_positions_visited':new_previous_positions_visited})
                        print("Whoa we won")
                        print(new_previous_positions_visited)
                        break
                    new_previous_positions_visited = previous_positions_visited.copy()
                    new_previous_positions_visited.append(new_coordinates_for_die_might_be)   
                    child_list.append({'current_move':new_current_move, 'current_sum':new_current_sum, 'die_touching_surface_coordi':new_coordinates_for_die_might_be, 'die_faces_configuration':new_die_faces_configuration, 'current_numbers_assigned_to_faces':new_current_numbers_assigned_to_faces, 'previous_positions_visited':new_previous_positions_visited})
                    
                elif new_current_sum != values:
                    continue
                else:
                    print("logic error")

    if child_list == []:
        break
    parent_list = child_list   

Whoa we won
[[5, 0], [4, 0], [4, 1], [4, 2], [5, 2], [5, 1], [4, 1], [3, 1], [2, 1], [1, 1], [0, 1], [0, 2], [1, 2], [1, 1], [1, 0], [2, 0], [2, 1], [2, 2], [3, 2], [3, 3], [4, 3], [5, 3], [5, 4], [5, 5], [4, 5], [3, 5], [3, 4], [3, 3], [2, 3], [1, 3], [1, 4], [1, 5], [0, 5]]


In [42]:
repeated_removed_list = []
total_places_visited = 0
for i in parent_list[0]['previous_positions_visited']:
    if i not in repeated_removed_list:
        total_places_visited += numbers[i[0]][i[1]]
        repeated_removed_list.append(i)

In [43]:
total_of_all_places = 0
for i in numbers:
    for j in i:
        total_of_all_places += j
total_of_all_places

9767

In [44]:
total_of_all_places - total_places_visited

1935